### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load 
file_to_load = "Resources/purchasedata.csv"

# Read Purchasing File and store into Pandas data frame, 
# print head to make sure it prints and for header reference
# I was having trouble with my sub data sets, so I just moved all their definitions up here so 
# they weren't getting overwritten in prior steps.  
df = pd.read_csv(file_to_load)
binning_df = df.copy()
binDupe = df.drop_duplicates(subset = 'SN', keep = 'first')
groupBySN = df.groupby(["SN"])
groupItem = df.groupby(["Item ID", "Item Name"])
df.head()


## Player Count

* Display the total number of players


In [ ]:
uniquePlayers = len(df['SN'].unique())

pd.DataFrame({"Total Players":[uniquePlayers]})


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [ ]:
#Filter out duplicate items
uniqueItems = len(df["Item Name"].unique())
# Calculate average sale price
averagePrice = df["Price"].mean()
# Count how many purchases were made
countPurchases = len(df["Price"])
# Calculate total revenue (Sum of all purchases)
totalPurchases = df["Price"].sum()
summaryTable = pd.DataFrame({"Unique Items":[uniqueItems],"Average Price":[averagePrice],"Number of Purchases":[countPurchases], "Total Purchases":[totalPurchases]})
summaryTable

## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [ ]:
# Count number of each gender
# First have to drop the duplicates 
dropDupes = df.drop_duplicates(subset = 'SN', keep = 'first')
# This gives the total non-duplicate players
genderCount = dropDupes["Gender"].count()

#Count each gender
maleCount = dropDupes['Gender'].value_counts()['Male']
femaleCount = dropDupes['Gender'].value_counts()['Female']
otherCount = dropDupes['Gender'].value_counts()['Other / Non-Disclosed']

#Calculate the percentage of each gender
malePercent = (maleCount / genderCount) * 100
femalePercent = (femaleCount / genderCount) * 100
otherPercent = (otherCount / genderCount) * 100

genderDemo = pd.DataFrame({'':['Male','Female','Other / Non-Disclosed'], 
                           'Percentage of Players':[malePercent, femalePercent, otherPercent],
                           'Total Count':[maleCount, femaleCount, otherCount]})

genderDemo["Percentage of Players"] = genderDemo["Percentage of Players"].map("{:.2f}%".format)
genderDemo = genderDemo.set_index('')
genderDemo



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [ ]:
groupedDF = df.groupby(['Gender'])
purchaseCount = groupedDF['SN'].count()
avgPurchasePrice = groupedDF['Price'].mean()
purchaseValue = groupedDF['Price'].sum()

groupedDeDuped = dropDupes.groupby(['Gender'])

purchNorm = (groupedDF['Price'].sum()/groupedDF['SN'].count())

PurchasedByGen = pd.DataFrame({'Purchase Count': purchaseCount, 'Avg Purchase Price': avgPurchasePrice,
                               'Normalized Totals': purchNorm})
PurchasedByGen

## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [ ]:
#Define bins and bin labels
ageBins = [0, 10, 14, 18, 22, 26, 30, 34, 38, 40]
binLabel = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '>45']

#Count how many in each Age Range
df['Age Range']=pd.cut(df['Age'],ageBins,labels=binLabel)
ageDemoTotal=df['Age Range'].value_counts()

#Calculate Percent of each age range
ageDemoPercent=ageDemoTotal/uniquePlayers *100

#Data Frame for Age demographics 
ageSummary=pd.DataFrame({'Percent of Players':ageDemoPercent,'Total Count':ageDemoTotal})
df=pd.DataFrame({'Percent of Players':ageDemoPercent,'Total Count':ageDemoTotal})
ageSummary_df=df.loc[:,['Percent of Players','Total Count']]

ageSummary_df=ageSummary_df.sort_index()
ageSummary_df.round(2)



## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [ ]:
#binning_df = df.copy()  <-- moved this up top
binning_df['Age Groups'] = pd.cut(binning_df['Age'], ageBins, labels=binLabel)


binColumn = pd.cut(binning_df["Age"], ageBins, labels=binLabel)
groupedBin = binning_df.groupby(["Age Groups"])

#Calculate Purchases (#), Average Price and Total spent
binPurchCount = groupedBin['Age'].count()
binPurchAvg = groupedBin['Price'].mean()
binPurchTotal = groupedBin['Price'].sum()

# Create a new, clean data frame for bin manipulation
binDupe['Age Groups'] = pd.cut(binDupe['Age'], ageBins, labels = binLabel)


binDupe = binDupe.groupby(['Age Groups'])

binNormal = (groupedBin['Price'].sum() / binDupe['SN'].count())
ageDemographic = pd.DataFrame({'Purchase Count': binPurchCount, 'Average Purchase Price': binPurchAvg,
                              'Total Purchase Value': binPurchTotal, 'Normalized Totals':binNormal})

ageDemographic["Average Purchase Price"] = ageDemographic["Average Purchase Price"].map("${:.2f}".format)
ageDemographic["Total Purchase Value"] = ageDemographic["Total Purchase Value"].map("${:.2f}".format)
ageDemographic["Normalized Totals"] = ageDemographic["Normalized Totals"].map("${:.2f}".format)
ageDemographic = ageDemographic[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]
ageDemographic.head(12)

# I don't understand the warning message that is generated with this block.  Luckily I get the correct answer

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [ ]:
#groupBySN = df.groupby(["SN"])  <--moved up top
groupSNCount = groupBySN["Item ID"].count()
groupSNTotal = groupBySN["Price"].sum()
groupSNAvg = (groupSNTotal / groupSNCount)

# Build DF and format
spenders = pd.DataFrame({"Purchase Count": groupSNCount,
                         "Average Purchase Price": groupSNAvg,
                         "Total Purchase Value": groupSNTotal})

spenders = spenders.sort_values("Total Purchase Value", ascending=False) 
spenders["Average Purchase Price"] = spenders["Average Purchase Price"].map("${:.2f}".format)
spenders["Total Purchase Value"] = spenders["Total Purchase Value"].map("${:.2f}".format)
spenders = spenders[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]
spenders.head() 

## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [ ]:
#groupItem = df.groupby(["Item ID", "Item Name"])  <-- moved up top
groupItemCount = groupItem["SN"].count()
groupPriceSum = groupItem["Price"].sum()
groupItemPrice = (groupPriceSum / groupItemCount)
groupItemValue = (groupItemPrice * groupItemCount)

# New DF with formatting
popularItem = pd.DataFrame({"Purchase Count": groupItemCount,
                          "Item Price": groupItemPrice,
                          "Total Purchase Value": groupItemValue})

popularItem = popularItem.sort_values("Purchase Count", ascending=False) 
popularItem["Item Price"] = popularItem["Item Price"].map("${:.2f}".format)
popularItem["Total Purchase Value"] = popularItem["Total Purchase Value"].map("${:.2f}".format)
popularItem = popularItem[["Purchase Count", "Item Price", "Total Purchase Value"]]
popularItem.head()

## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [ ]:
#groupedItem = df.groupby(["Item ID", "Item Name"])  <-- moved up top
groupItemCount = groupItem["Gender"].count()
groupSum = groupItem["Price"].sum()
groupItemPrice = (groupSum / groupItemCount)

# Make a new DF and format
mostProf = pd.DataFrame({"Purchase Count": groupItemCount,
                          "Item Price": groupItemPrice,
                          "Total Purchase Value": groupSum})

mostProf = mostProf.sort_values("Total Purchase Value", ascending=False) 
mostProf["Item Price"] = mostProf["Item Price"].map("${:.2f}".format)
mostProf["Total Purchase Value"] = mostProf["Total Purchase Value"].map("${:.2f}".format)
mostProf = mostProf[["Purchase Count", "Item Price", "Total Purchase Value"]]
mostProf.head()

In [ ]:
#Observations:  
# 1) Gaming is heavily male oriented at 84+%, compared to the combined female/other/non-dislcosed of 15$
# 2) The 15 to 30 demographic are by far the least patient/istant gratification, 
#    and are willing to buy skills/tools rather than earn them in game play
# 3) This game is most popular with the 15-30 demo, which should influence where marketing dollars are spent.